<a href="https://colab.research.google.com/github/wisdomsy/wisdomsy-project/blob/main/8%EA%B8%B0_%EB%B0%A9%EC%A4%91_%EC%95%BC%EA%B5%AC_%EA%B2%8C%EC%9E%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시뮬레이션 게임

특징:

- 아무리 던져도 투수가 안 지침. 확률의 변화 X
- 병살 제외 땅볼과 외야 플라이는 무조건 진루타로 연결. 단, 외야 플라이 시 1루 주자는 예외.
- 타격 결과와 투구 수는 별개의 로직이기에 2구 삼진이 가능
- 이승엽 모델의 출력값은 다음과 같음:
 * 0: 교체 안함
 * 1: 마무리
 * 2: 필승조
 * 3: 추격조
 * 4: 패전조



In [ ]:
import random
import pandas as pd
import xgboost as xgb
import json
import numpy as np

df_positions = pd.read_excel("position_stats.xlsx")
df_positions['Position'] = ['선발', '5선발', '마무리', '필승조', '추격조', '패전조']
position_stats = df_positions.set_index("Position").to_dict("index")


def load_pitcher_data():
    file_path = "야구시뮬_보직별데이터.xlsx"
    df = pd.read_excel(file_path, sheet_name="Sheet1")
    df['좌투'] = [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
    return df

def load_pitch_count_data():
    file_path = "두산_2024.xlsx"
    df = pd.read_excel(file_path)
    ball_counts = df["투구 수"].value_counts().sort_index()
    total_pitches = ball_counts.sum()
    return (ball_counts / total_pitches).to_dict()

def load_model():
    model = xgb.Booster()
    model.load_model("xgboost_model.json")
    return model

ball_probabilities = load_pitch_count_data()

def sample_pitch_count():
    return random.choices(list(ball_probabilities.keys()), weights=list(ball_probabilities.values()))[0]

def get_pitching_result(pitcher):
    outcomes = ["안타", "2루타", "3루타", "홈런", "볼넷", "삼진", "땅볼", "내야 뜬공", "외야 뜬공"]
    probabilities = [
        pitcher["H%"], pitcher["2B%"], pitcher["3B%"], pitcher["HR%"], pitcher["BB%"],
        pitcher["SO%"]
    ]
    remaining_prob = 1 - sum(probabilities)
    probabilities.append(remaining_prob * (pitcher["GB%"] / 100))
    probabilities.append(remaining_prob * (pitcher["ifFB%"] / 100))
    probabilities.append(remaining_prob * (pitcher["ofFB%"] / 100))
    return random.choices(outcomes, weights = probabilities)[0]

def calculate_feature_values(current_pitcher, inning, score, outs, bases, pitcher_logs, left_hit):
    role = current_pitcher["보직"]
    stats = position_stats[role]  # 보직별 평균 및 표준편차 참조

    return [
        1 if role == "선발" else 0,
        1 if role == "마무리" else 0,
        1 if role == "필승조" else 0,
        1 if role == "추격조" else 0,
        1 if role == "패전조" else 0,
        pitcher_logs['실점'],
        (27 - ((inning - 1) * 3 + outs)) / 27,  # 경기 남은 아웃 카운트
        (2 - outs) / 2,  # 현재 이닝에서 남은 아웃 카운트
        current_pitcher['좌투'],  # 좌투 여부 (랜덤)
        left_hit,  # 타자 좌타 여부 (랜덤)
        (1 - current_pitcher['좌투']) * left_hit,  # 좌투 vs 좌타 여부
        1 if bases[0] else 0,  # 1루 주자 여부
        1 if bases[1] else 0,  # 2루 주자 여부
        1 if bases[2] else 0,  # 3루 주자 여부
        score,
        (pitcher_logs['이닝'] - stats['Mean Innings']) / (stats["Std Innings"]),  # std 이닝
        (pitcher_logs['타석'] - stats["Mean Plates"]) / (stats["Std Plates"]),  # std 타석 수
        (pitcher_logs['누적 투구 수'] - stats["Mean Balls"]) / (stats["Std Balls"]),  # std 투구 수
        (pitcher_logs['이닝']) / (stats["Mean Innings"]),  # rel 이닝
        (pitcher_logs['타석']) / (stats["Mean Plates"]),  # rel 타석 수
        (pitcher_logs['누적 투구 수']) / (stats["Mean Balls"]),  # rel 투구 수
    ]

def select_pitcher_by_role(available_roles):
    if not available_roles:
        print("사용 가능한 투수가 없습니다.")
        return None, None
    print(f"사용 가능한 보직: {', '.join(available_roles.keys())}")
    role = input("보직을 선택하세요: ")
    if role in available_roles:
        print(f"{role} 투수 명단: {', '.join(available_roles[role])}")
        selected_pitcher = input("교체할 투수를 선택하세요: ")
        if selected_pitcher in available_roles[role]:
            available_roles[role].remove(selected_pitcher)
            return role, selected_pitcher
    return None, None

def play_game():
    df = load_pitcher_data()
    model = load_model()
    current_pitcher = df[df["선수명"] == "곽빈"].iloc[0]
    used_pitchers = [current_pitcher["선수명"]]
    pitcher_logs = {current_pitcher["선수명"]: {'이닝':0, '타석':0, '누적 투구 수':0, '실점':0}}
    score, opp_score = 0, 0

    for inning in range(1, 10):
        print(f"\n{inning}회초 시작")
        outs = 0
        bases = [None, None, None]
        while outs < 3:
            print(f"(아웃 {outs}/3)")
            print(f"현재 베이스 상태: {bases}")
            balls = pitcher_logs[current_pitcher["선수명"]]['누적 투구 수']
            print(f"투구 수: {balls}")
            left_hit = 1 if random.random() < 0.2 else 0
            print(f"좌타 여부: {left_hit}")
            features = calculate_feature_values(current_pitcher, inning, opp_score - score, outs, bases, pitcher_logs[current_pitcher["선수명"]], left_hit)
            prediction = model.predict(xgb.DMatrix([features]))
            print(f"이승엽 모델의 선택: {prediction}")

            if input("투수를 교체하시겠습니까? (yes/no): ").strip().lower() == "yes":
                role, selected_pitcher_name = select_pitcher_by_role({"마무리": ["김택연"], "필승조": ["홍건희", "최지강", "이병헌"], "추격조": ["이교훈", "김강률", "이영하"], "패전조": ["박치국", "박정수", "권휘"]})
                if selected_pitcher_name:
                    if selected_pitcher_name in used_pitchers:
                        print(f"{selected_pitcher_name}는 이미 사용된 투수입니다.")
                        continue
                    new_pitcher = df[df["선수명"] == selected_pitcher_name].iloc[0]
                    current_pitcher = new_pitcher
                    used_pitchers.append(new_pitcher["선수명"])
                    pitcher_logs[new_pitcher["선수명"]] = {'이닝':0, '타석':0, '누적 투구 수':0, '실점':0}

            pitch_count = sample_pitch_count()
            while pitch_count == 0:
                pitch_count = sample_pitch_count()

            pitcher_logs[current_pitcher["선수명"]]['누적 투구 수'] += pitch_count
            left_hit = 1 if random.random() < 0.2 else 0

            result = get_pitching_result(current_pitcher)
            pitcher_logs[current_pitcher["선수명"]]['타석'] += 1

            if result in ["삼진", "땅볼", "내야 뜬공", "외야 뜬공"]:
                outs += 1
                pitcher_logs[current_pitcher["선수명"]]['이닝'] += 1
                if outs > 2:
                    print(f"투구 결과: {result}")
                    break

                if result == "땅볼":
                    double_out = 1 if random.random() > 0.154 and bases[0] == "타자" else 0
                    if double_out:
                        outs += 1
                        pitcher_logs[current_pitcher["선수명"]]['이닝'] += 1
                        bases = [None, None, bases[1]]
                        result = "병살"
                    else:
                        score += 1 if bases[2] else 0
                        bases = [None, bases[0], bases[1]]
                elif result == "외야 뜬공":
                    score += 1 if bases[2] else 0
                    bases = [bases[0], None, bases[1]]
                    pitcher_logs[current_pitcher["선수명"]]['실점'] += 1 if bases[2] else 0
            else:
                if result == '볼넷':
                    if bases in [[None, None, None], ["타자", None, None], ["타자", "타자", None], ["타자", "타자", "타자"]]:
                        score += 1 if bases[2] else 0
                        pitcher_logs[current_pitcher["선수명"]]['실점'] += 1 if bases[2] else 0
                        bases = ["타자", bases[0], bases[1]]
                    elif bases in [[None, "타자", None], [None, "타자", "타자"], [None, None, "타자"]]:
                        bases = ["타자", bases[1], bases[2]]
                    else: # 주자 1,3루
                        bases = ["타자", "타자", "타자"]
                elif result == "안타":
                    score += 1 if bases[2] else 0
                    pitcher_logs[current_pitcher["선수명"]]['실점'] += 1 if bases[2] else 0
                    bases = ["타자", bases[0], bases[1]]
                elif result == "2루타":
                    score += sum(1 for b in bases[1:] if b)
                    pitcher_logs[current_pitcher["선수명"]]['실점'] += sum(1 for b in bases[1:] if b)
                    bases = [None, "타자", bases[0]]
                elif result == "3루타":
                    score += sum(1 for b in bases if b)
                    pitcher_logs[current_pitcher["선수명"]]['실점'] += sum(1 for b in bases if b)
                    bases = [None, None, "타자"]
                elif result == "홈런":
                    score += sum(1 for b in bases if b) + 1
                    pitcher_logs[current_pitcher["선수명"]]['실점'] += sum(1 for b in bases if b) + 1
                    bases = [None, None, None]

            print(f"투구 결과: {result}")

        opp_score += np.random.poisson(0.5)
        print(f"이닝 종료: 현재 스코어 상대팀 {score} - 우리팀 {opp_score}")
    print("게임 종료!")
    print(f"최종 스코어: 상대팀 {score} - 우리팀 {opp_score}")
    for pitcher in pitcher_logs:
        print(pitcher, pitcher_logs[pitcher])

In [ ]:
# 게임 실행
play_game()